In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_6_6_0,0.925406,0.896078,0.748832,0.849756,0.013955,0.019464,0.031478,0.025117,"Hidden Size=[30, 15], regularizer=0.02, learni..."
28,model_10_5_5,0.843879,0.812278,0.644702,0.731523,0.029207,0.033693,0.070427,0.050980,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
29,model_10_5_6,0.840999,0.794810,0.555086,0.678757,0.029745,0.036828,0.088190,0.060999,"Hidden Size=[30, 15, 7], regularizer=0.02, lea..."
30,model_5_6_5,0.836684,0.853953,0.766287,0.801627,0.030553,0.024549,0.065448,0.043796,"Hidden Size=[30, 15], regularizer=0.02, learni..."
31,model_5_6_6,0.836468,0.840907,0.756826,0.790724,0.030593,0.026742,0.068097,0.046203,"Hidden Size=[30, 15], regularizer=0.02, learni..."
...,...,...,...,...,...,...,...,...,...,...
242,model_5_7_2,0.751991,0.889973,0.921325,0.909717,0.046397,0.015574,0.019731,0.017530,"Hidden Size=[30, 15], regularizer=0.02, learni..."
243,model_5_7_11,0.751893,0.811405,0.622497,0.697768,0.046415,0.026694,0.094672,0.058684,"Hidden Size=[30, 15], regularizer=0.02, learni..."
244,model_5_7_7,0.751817,0.853434,0.736399,0.783217,0.046429,0.020745,0.066107,0.042092,"Hidden Size=[30, 15], regularizer=0.02, learni..."
253,model_5_7_10,0.750994,0.821139,0.644561,0.714935,0.046583,0.025317,0.089139,0.055350,"Hidden Size=[30, 15], regularizer=0.02, learni..."
